In [ ]:
# Código para consulta de quais budgets estão criados a partir do tenancy especificado no arquivo de configuracao 

import oci
import sys

# Default config file and profile
config = oci.config.from_file(file_location="C:\\Users\\rodrigo.almeida\\Documents\\GitHub\\BudgetReports\\.oci\\config")
budget_client = oci.budget.BudgetClient(config)

print(config)

# The first argument is the name of the script, so start the index at 1
compartment_id = config['tenancy']


print("Compartment Id: " + compartment_id)


# list all budgets
budgets = oci.pagination.list_call_get_all_results(
    budget_client.list_budgets,
    compartment_id
).data
print('ListBudgets for compartment with OCID: {}'.format(compartment_id))
for budget in budgets:
    print(budget)


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

file = "..\data\huawei\dataprev_CostAnalysis_LinkedAccount_ServiceType_EnterpriseProjects_20250121111903956_0001.csv"
# penguins["Service Type"].value_counts().index
data = pd.read_csv(file)
data = data.drop(columns=["Linked Account","Enterprise Projects","Total"])
data = data.drop(0)
data = data.transpose()
data = data.set_axis(data.iloc[0], axis=1)
data.drop(index=data.index[0], axis=0, inplace=True)
# data.head()
print(data.iloc[:,0])
# print()
# print(data["Linked Account"].value_counts().index)
# print(data["Enterprise Projects"].value_counts().index)

# 
# df_transposed = data.transpose()
# print(df_transposed) 
# col = df_transposed.columns
# print(len(col))



In [2]:
import seaborn as sns
import pandas as pd
file = "..\data\huawei\dataprev_CostAnalysis_LinkedAccount_ServiceType_EnterpriseProjects_20250121111903956_0001.csv"
# penguins["Service Type"].value_counts().index
data = pd.read_csv(file)
data = data.drop(columns=["Linked Account","Enterprise Projects","Total"])
data = data.drop(0)
data = data.transpose()
data = data.set_axis(data.iloc[0], axis=1)
data.drop(index=data.index[0], axis=0, inplace=True)

data.head()

df = pd.DataFrame({'date': data['Service Type'],
                   'Cosumo': data['aPaaS Deterministic O&M (aPaaSDeterministicOM) (USD)'],
                   'Servico': ['aPaaS Deterministic O&M (aPaaSDeterministicOM) (USD)']})

#plot multiple time series
sns.lineplot(x='date', y='Cosumo', hue='Servico', data=df)

# print(df_transposed)
# sns.histplot(data=data, x="Total", hue="Service Type")


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\rodrigo.almeida\AppData\Local\Temp\ipykernel_8108\1984584581.py:2: SyntaxWarning: invalid escape sequence '\d'
  file = "..\data\huawei\dataprev_CostAnalysis_LinkedAccount_ServiceType_EnterpriseProjects_20250121111903956_0001.csv"
C:\Users\rodrigo.almeida\AppData\Local\Temp\ipykernel_8108\1984584581.py:2: SyntaxWarning: invalid escape sequence '\d'
  file = "..\data\huawei\dataprev_CostAnalysis_LinkedAccount_ServiceType_EnterpriseProjects_20250121111903956_0001.csv"


NameError: name 'pd' is not defined

In [3]:
import seaborn as sns
import pandas as pd
file = "usage_compartments.csv"
# penguins["Service Type"].value_counts().index
data = pd.read_csv(file)

data.head()


,Compartment Path,Service,Start Date,End Date,Days,Currency,Quantity,Osr Eligible Cost,Cost,Month-31,Year
0,ritm0021660bilhetagemapi/BilhetagemAPI_DIPP,Analytics,01/01/2025,01/20/2025,20,BRL,740.8,7982.2,7982.2,12372,145675
1,ritm0021660bilhetagemapi/BilhetagemAPI_DIPP,Database,01/01/2025,01/20/2025,20,BRL,1411.7,1278.6,1278.6,1982,23334
2,ritm0021660bilhetagemapi/BilhetagemAPI_DIPP,Object Storage,01/01/2025,01/20/2025,20,BRL,17.5,1.0,1.0,1,17
3,ritm0023258pococrinssdiia/poc-ocr-atestado-medico,Data Science,01/01/2025,01/20/2025,20,BRL,9258.8,266.7,266.7,413,4867
4,ritm0023258pococrinssdiia/poc-ocr-atestado-medico,Object Storage,01/01/2025,01/20/2025,20,BRL,0.0,0.0,0.0,0,0
